In [4]:
#Genetic algorithm for hyperparameter tuning of a deep Neural Network using Tensorflow

In [2]:
import tensorflow as tf
import operator
he_init = tf.contrib.layers.variance_scaling_initializer()

def dnn(inputs, n_hidden_layers=5, n_neurons=100, name=None,
        activation=tf.nn.elu, initializer=he_init):
    with tf.variable_scope(name, "dnn"):
        for layer in range(n_hidden_layers):
            inputs = tf.layers.dense(inputs, n_neurons, activation=activation,
                                     kernel_initializer=initializer,
                                     name="hidden%d" % (layer + 1))
        return inputs

In [3]:
n_inputs = 28 * 28 # MNIST
n_outputs = 5

#reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

dnn_outputs = dnn(X)

logits = tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=he_init, name="logits")
Y_proba = tf.nn.softmax(logits, name="Y_proba")

In [4]:
learning_rate = 0.01

xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy, name="loss")

optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss, name="training_op")

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [5]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [6]:
X_train1 = mnist.train.images[mnist.train.labels < 5]
y_train1 = mnist.train.labels[mnist.train.labels < 5]
X_valid1 = mnist.validation.images[mnist.validation.labels < 5]
y_valid1 = mnist.validation.labels[mnist.validation.labels < 5]
X_test1 = mnist.test.images[mnist.test.labels < 5]
y_test1 = mnist.test.labels[mnist.test.labels < 5]

In [7]:
import numpy as np

n_epochs = 1000
batch_size = 20

max_checks_without_progress = 10
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train1))
        for rnd_indices in np.array_split(rnd_idx, len(X_train1) // batch_size):
            X_batch, y_batch = X_train1[rnd_indices], y_train1[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid1, y: y_valid1})
        if loss_val < best_loss:
            save_path = saver.save(sess, "./my_mnist_model_0_to_4.ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    saver.restore(sess, "./my_mnist_model_0_to_4.ckpt")
    acc_test = accuracy.eval(feed_dict={X: X_test1, y: y_test1})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

0	Validation loss: 0.123383	Best loss: 0.123383	Accuracy: 97.11%
1	Validation loss: 0.177884	Best loss: 0.123383	Accuracy: 95.19%
2	Validation loss: 0.109575	Best loss: 0.109575	Accuracy: 97.97%
3	Validation loss: 0.092251	Best loss: 0.092251	Accuracy: 97.62%
4	Validation loss: 2.287983	Best loss: 0.092251	Accuracy: 70.76%
5	Validation loss: 0.681341	Best loss: 0.092251	Accuracy: 77.80%
6	Validation loss: 1.424878	Best loss: 0.092251	Accuracy: 57.08%
7	Validation loss: 1.636677	Best loss: 0.092251	Accuracy: 18.73%
8	Validation loss: 1.707939	Best loss: 0.092251	Accuracy: 22.01%
9	Validation loss: 1.626957	Best loss: 0.092251	Accuracy: 18.73%
10	Validation loss: 1.623782	Best loss: 0.092251	Accuracy: 19.08%
11	Validation loss: 1.680743	Best loss: 0.092251	Accuracy: 22.01%
12	Validation loss: 1.697856	Best loss: 0.092251	Accuracy: 20.91%
13	Validation loss: 1.626846	Best loss: 0.092251	Accuracy: 19.08%
14	Validation loss: 1.661726	Best loss: 0.092251	Accuracy: 18.73%
15	Validation loss: 

In [8]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.exceptions import NotFittedError

class DNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, n_hidden_layers=5, n_neurons=100, optimizer_class=tf.train.AdamOptimizer,
                 learning_rate=0.01, batch_size=20, activation=tf.nn.elu, initializer=he_init,
                 batch_norm_momentum=None, dropout_rate=None, random_state=None):
        """Initialize the DNNClassifier by simply storing all the hyperparameters."""
        self.n_hidden_layers = n_hidden_layers
        self.n_neurons = n_neurons
        self.optimizer_class = optimizer_class
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.activation = activation
        self.initializer = initializer
        self.batch_norm_momentum = batch_norm_momentum
        self.dropout_rate = dropout_rate
        self.random_state = random_state
        self._session = None

    def _dnn(self, inputs):
        """Build the hidden layers, with support for batch normalization and dropout."""
        for layer in range(self.n_hidden_layers):
            if self.dropout_rate:
                inputs = tf.layers.dropout(inputs, self.dropout_rate, training=self._training)
            inputs = tf.layers.dense(inputs, self.n_neurons,
                                     kernel_initializer=self.initializer,
                                     name="hidden%d" % (layer + 1))
            if self.batch_norm_momentum:
                inputs = tf.layers.batch_normalization(inputs, momentum=self.batch_norm_momentum,
                                                       training=self._training)
            inputs = self.activation(inputs, name="hidden%d_out" % (layer + 1))
        return inputs

    def _build_graph(self, n_inputs, n_outputs):
        """Build the same model as earlier"""
        if self.random_state is not None:
            tf.set_random_seed(self.random_state)
            np.random.seed(self.random_state)

        X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
        y = tf.placeholder(tf.int32, shape=(None), name="y")

        if self.batch_norm_momentum or self.dropout_rate:
            self._training = tf.placeholder_with_default(False, shape=(), name='training')
        else:
            self._training = None

        dnn_outputs = self._dnn(X)

        logits = tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=he_init, name="logits")
        Y_proba = tf.nn.softmax(logits, name="Y_proba")

        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                                  logits=logits)
        loss = tf.reduce_mean(xentropy, name="loss")

        optimizer = self.optimizer_class(learning_rate=self.learning_rate)
        training_op = optimizer.minimize(loss)

        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

        init = tf.global_variables_initializer()
        saver = tf.train.Saver()

        # Make the important operations available easily through instance variables
        self._X, self._y = X, y
        self._Y_proba, self._loss = Y_proba, loss
        self._training_op, self._accuracy = training_op, accuracy
        self._init, self._saver = init, saver

    def close_session(self):
        if self._session:
            self._session.close()

    def _get_model_params(self):
        """Get all variable values (used for early stopping, faster than saving to disk)"""
        with self._graph.as_default():
            gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
        return {gvar.op.name: value for gvar, value in zip(gvars, self._session.run(gvars))}

    def _restore_model_params(self, model_params):
        """Set all variables to the given values (for early stopping, faster than loading from disk)"""
        gvar_names = list(model_params.keys())
        assign_ops = {gvar_name: self._graph.get_operation_by_name(gvar_name + "/Assign")
                      for gvar_name in gvar_names}
        init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
        feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
        self._session.run(assign_ops, feed_dict=feed_dict)

    def fit(self, X, y, n_epochs=100, X_valid=None, y_valid=None):
        """Fit the model to the training set. If X_valid and y_valid are provided, use early stopping."""
        self.close_session()

        # infer n_inputs and n_outputs from the training set.
        n_inputs = X.shape[1]
        self.classes_ = np.unique(y)
        n_outputs = len(self.classes_)
        
        # Translate the labels vector to a vector of sorted class indices, containing
        # integers from 0 to n_outputs - 1.
        # For example, if y is equal to [8, 8, 9, 5, 7, 6, 6, 6], then the sorted class
        # labels (self.classes_) will be equal to [5, 6, 7, 8, 9], and the labels vector
        # will be translated to [3, 3, 4, 0, 2, 1, 1, 1]
        self.class_to_index_ = {label: index
                                for index, label in enumerate(self.classes_)}
        y = np.array([self.class_to_index_[label]
                      for label in y], dtype=np.int32)
        
        self._graph = tf.Graph()
        with self._graph.as_default():
            self._build_graph(n_inputs, n_outputs)
            # extra ops for batch normalization
            extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

        # needed in case of early stopping
        max_checks_without_progress = 20
        checks_without_progress = 0
        best_loss = np.infty
        best_params = None
        
        # Now train the model!
        self._session = tf.Session(graph=self._graph)
        with self._session.as_default() as sess:
            self._init.run()
            for epoch in range(n_epochs):
                rnd_idx = np.random.permutation(len(X))
                for rnd_indices in np.array_split(rnd_idx, len(X) // self.batch_size):
                    X_batch, y_batch = X[rnd_indices], y[rnd_indices]
                    feed_dict = {self._X: X_batch, self._y: y_batch}
                    if self._training is not None:
                        feed_dict[self._training] = True
                    sess.run(self._training_op, feed_dict=feed_dict)
                    if extra_update_ops:
                        sess.run(extra_update_ops, feed_dict=feed_dict)
                if X_valid is not None and y_valid is not None:
                    loss_val, acc_val = sess.run([self._loss, self._accuracy],
                                                 feed_dict={self._X: X_valid,
                                                            self._y: y_valid})
                    if loss_val < best_loss:
                        best_params = self._get_model_params()
                        best_loss = loss_val
                        checks_without_progress = 0
                    else:
                        checks_without_progress += 1
                    print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
                        epoch, loss_val, best_loss, acc_val * 100))
                    if checks_without_progress > max_checks_without_progress:
                        print("Early stopping!")
                        break
                else:
                    loss_train, acc_train = sess.run([self._loss, self._accuracy],
                                                     feed_dict={self._X: X_batch,
                                                                self._y: y_batch})
                    print("{}\tLast training batch loss: {:.6f}\tAccuracy: {:.2f}%".format(
                        epoch, loss_train, acc_train * 100))
            # If we used early stopping then rollback to the best model found
            if best_params:
                self._restore_model_params(best_params)
            return self

    def predict_proba(self, X):
        if not self._session:
            raise NotFittedError("This %s instance is not fitted yet" % self.__class__.__name__)
        with self._session.as_default() as sess:
            return self._Y_proba.eval(feed_dict={self._X: X})

    def predict(self, X):
        class_indices = np.argmax(self.predict_proba(X), axis=1)
        return np.array([[self.classes_[class_index]]
                         for class_index in class_indices], np.int32)

    def save(self, path):
        self._saver.save(self._session, path)

In [9]:
dnn_clf = DNNClassifier(random_state=42)
dnn_clf.fit(X_train1, y_train1, n_epochs=1000, X_valid=X_valid1, y_valid=y_valid1)

0	Validation loss: 0.190826	Best loss: 0.190826	Accuracy: 96.64%
1	Validation loss: 1.689649	Best loss: 0.190826	Accuracy: 18.73%
2	Validation loss: 1.660114	Best loss: 0.190826	Accuracy: 20.91%
3	Validation loss: 1.778077	Best loss: 0.190826	Accuracy: 22.01%
4	Validation loss: 1.667106	Best loss: 0.190826	Accuracy: 22.01%
5	Validation loss: 1.654532	Best loss: 0.190826	Accuracy: 22.01%
6	Validation loss: 1.680933	Best loss: 0.190826	Accuracy: 18.73%
7	Validation loss: 1.779077	Best loss: 0.190826	Accuracy: 22.01%
8	Validation loss: 1.699482	Best loss: 0.190826	Accuracy: 19.27%
9	Validation loss: 1.767771	Best loss: 0.190826	Accuracy: 20.91%
10	Validation loss: 1.629350	Best loss: 0.190826	Accuracy: 22.01%
11	Validation loss: 1.812643	Best loss: 0.190826	Accuracy: 22.01%
12	Validation loss: 1.675939	Best loss: 0.190826	Accuracy: 18.73%
13	Validation loss: 1.633259	Best loss: 0.190826	Accuracy: 20.91%
14	Validation loss: 1.652904	Best loss: 0.190826	Accuracy: 20.91%
15	Validation loss: 

DNNClassifier(activation=<function elu at 0x00000143F62CC400>,
       batch_norm_momentum=None, batch_size=20, dropout_rate=None,
       initializer=<function variance_scaling_initializer.<locals>._initializer at 0x00000143F8607D08>,
       learning_rate=0.01, n_hidden_layers=5, n_neurons=100,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42)

In [10]:
from sklearn.metrics import accuracy_score

y_pred = dnn_clf.predict(X_test1)
accuracy_score(y_test1, y_pred)

0.97081144191476942

In [21]:
"""
def getCost(n_hidden_layers, n_neurons, optimizer_class,
                 learning_rate, batch_size, activation, initializer,
                 batch_norm_momentum, dropout_rate, random_state,
                   X_train1, y_train1,X_valid1,y_valid1,X_test1,y_test1,n_epochs):
    
    
    
    dnn_clf = DNNClassifier(n_hidden_layers, n_neurons, optimizer_class,
                 learning_rate, batch_size, activation, initializer,
                 batch_norm_momentum, dropout_rate, random_state)
    dnn_clf.fit(X_train1, y_train1, n_epochs, X_valid=X_valid1, y_valid=y_valid1)
    
    y_pred = dnn_clf.predict(X_test1)
    return accuracy_score(y_test1, y_pred)
"""

'\ndef getCost(n_hidden_layers, n_neurons, optimizer_class,\n                 learning_rate, batch_size, activation, initializer,\n                 batch_norm_momentum, dropout_rate, random_state,\n                   X_train1, y_train1,X_valid1,y_valid1,X_test1,y_test_1,n_epochs):\n    \n    \n    \n    dnn_clf = DNNClassifier(n_hidden_layers, n_neurons, optimizer_class,\n                 learning_rate, batch_size, activation, initializer,\n                 batch_norm_momentum, dropout_rate, random_state)\n    dnn_clf.fit(X_train1, y_train1, n_epochs, X_valid=X_valid1, y_valid=y_valid1)\n    \n    y_pred = dnn_clf.predict(X_test1)\n    return accuracy_score(y_test1, y_pred)\n'

In [11]:
from sklearn.model_selection import Randomize-

'\n#FOr running randomized algo\n#rnd_search = RandomizedSearchCV(DNNClassifier(random_state=42), param_distribs, n_iter=50,\n                                fit_params={"X_valid": X_valid1, "y_valid": y_valid1, "n_epochs": 1000},\n                                random_state=42, verbose=2)\n#rnd_search.fit(X_train1, y_train1)\n'

In [38]:
"""
from deap import base, creator, tools, algorithms
from scipy.stats import bernoulli

population_size = 4
num_generations = 4
gene_length = 10
params={'n_hidden_layers':[15,20],'n_neurons':[10,20]}

# As we are trying to minimize the RMSE score, that's why using -1.0. 
# In case, when you want to maximize accuracy for instance, use 1.0
creator.create('FitnessMax', base.Fitness, weights = (1.0,))
creator.create('Individual', list , fitness = creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register('binary', bernoulli.rvs, 0.5)
toolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.binary, 
n = gene_length)
toolbox.register('population', tools.initRepeat, list , toolbox.individual)

toolbox.register('mate', tools.cxOrdered)
toolbox.register('mutate', tools.mutShuffleIndexes, indpb = 0.6)
toolbox.register('select', tools.selRoulette)
toolbox.register('evaluate', getCost,X_train1,y_train1,X_valid1,y_valid1,X_test1,y_test1,params)

population = toolbox.population(n = population_size)
r = algorithms.eaSimple(population, toolbox, cxpb = 0.4, mutpb = 0.1, 
ngen = num_generations, verbose = False)
"""

"\nfrom deap import base, creator, tools, algorithms\nfrom scipy.stats import bernoulli\n\npopulation_size = 4\nnum_generations = 4\ngene_length = 10\nparams={'n_hidden_layers':[15,20],'n_neurons':[10,20]}\n\n# As we are trying to minimize the RMSE score, that's why using -1.0. \n# In case, when you want to maximize accuracy for instance, use 1.0\ncreator.create('FitnessMax', base.Fitness, weights = (1.0,))\ncreator.create('Individual', list , fitness = creator.FitnessMax)\n\ntoolbox = base.Toolbox()\ntoolbox.register('binary', bernoulli.rvs, 0.5)\ntoolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.binary, \nn = gene_length)\ntoolbox.register('population', tools.initRepeat, list , toolbox.individual)\n\ntoolbox.register('mate', tools.cxOrdered)\ntoolbox.register('mutate', tools.mutShuffleIndexes, indpb = 0.6)\ntoolbox.register('select', tools.selRoulette)\ntoolbox.register('evaluate', getCost,X_train1,y_train1,X_valid1,y_valid1,X_test1,y_test1,params)\n\npopul

In [176]:
#genetic algorithm implementation
generations=10


param={'n_hidden_layers':[5,10,15,20],'n_neurons':[10,20,30,15],'learning_rate':[0.0001,0.001,0.01,0.1,1]}


In [158]:
def generateRandomParams(param):
    param_list=[]
    for k,v in param_dict.items():
        rnd=np.random.randint(0,len(v))
        param_list.append(v[rnd])
        
    return param_list

In [159]:
param_list=generateRandomParams(param)

In [160]:
def getCost(X_train1, y_train1,X_valid1,y_valid1,X_test1,y_test1,param_list):
    
    
    n_hidden_layers=param_list[0]
    n_neurons=param_list[1]
    learning_rate=param_list=[2]
    
    dnn_clf = DNNClassifier(n_hidden_layers, n_neurons)
    dnn_clf.fit(X_train1, y_train1, n_epochs, X_valid=X_valid1, y_valid=y_valid1)
    
    y_pred = dnn_clf.predict(X_test1)
    return accuracy_score(y_test1, y_pred)

In [161]:
getCost(X_train1, y_train1,X_valid1,y_valid1,X_test1,y_test1,param_list)

0	Validation loss: 0.271155	Best loss: 0.271155	Accuracy: 92.77%
1	Validation loss: 0.213727	Best loss: 0.213727	Accuracy: 94.57%
2	Validation loss: 0.391432	Best loss: 0.213727	Accuracy: 91.95%


KeyboardInterrupt: 

In [162]:
#creates  a dictionary having values in descending order
def getTop_N_Fit(population):
    fitness_dict={}
    for i in range(population):
        param_list=generateRandomParams(param)
        #param_dict[i]=param_list
        fitness=getCost(X_train1, y_train1,X_valid1,y_valid1,X_test1,y_test1,param_list)
        fitness_dict[i]=(fitness,param_list)
        
    sorted_fitness_dict = dict(sorted(fitness_dict.items(), key=operator.itemgetter(1),reverse=True))
    return sorted_fitness_dict

In [164]:
sorted_fitness_dict=getTop_N_Fit(5)


0	Validation loss: 0.154623	Best loss: 0.154623	Accuracy: 96.17%
1	Validation loss: 0.364906	Best loss: 0.154623	Accuracy: 90.11%
2	Validation loss: 0.288331	Best loss: 0.154623	Accuracy: 91.20%
3	Validation loss: 0.328568	Best loss: 0.154623	Accuracy: 89.87%
4	Validation loss: 0.352195	Best loss: 0.154623	Accuracy: 88.58%
5	Validation loss: 0.432648	Best loss: 0.154623	Accuracy: 86.83%
6	Validation loss: 0.254245	Best loss: 0.154623	Accuracy: 92.69%
7	Validation loss: 0.332493	Best loss: 0.154623	Accuracy: 91.83%
8	Validation loss: 0.423323	Best loss: 0.154623	Accuracy: 88.55%
9	Validation loss: 0.653812	Best loss: 0.154623	Accuracy: 77.99%
10	Validation loss: 0.612142	Best loss: 0.154623	Accuracy: 78.69%
11	Validation loss: 0.710610	Best loss: 0.154623	Accuracy: 68.76%
12	Validation loss: 0.536184	Best loss: 0.154623	Accuracy: 82.21%
13	Validation loss: 0.329499	Best loss: 0.154623	Accuracy: 90.93%
14	Validation loss: 0.332647	Best loss: 0.154623	Accuracy: 91.95%
15	Validation loss: 

38	Validation loss: 1.047998	Best loss: 0.229844	Accuracy: 57.94%
39	Validation loss: 0.785590	Best loss: 0.229844	Accuracy: 64.74%
40	Validation loss: 0.767029	Best loss: 0.229844	Accuracy: 64.35%
41	Validation loss: 0.781985	Best loss: 0.229844	Accuracy: 64.97%
42	Validation loss: 0.822877	Best loss: 0.229844	Accuracy: 59.34%
43	Validation loss: 0.855906	Best loss: 0.229844	Accuracy: 58.17%
44	Validation loss: 0.801427	Best loss: 0.229844	Accuracy: 59.27%
45	Validation loss: 0.788450	Best loss: 0.229844	Accuracy: 59.93%
46	Validation loss: 0.793331	Best loss: 0.229844	Accuracy: 59.03%
47	Validation loss: 0.807582	Best loss: 0.229844	Accuracy: 58.84%
48	Validation loss: 0.808173	Best loss: 0.229844	Accuracy: 58.37%
49	Validation loss: 0.821813	Best loss: 0.229844	Accuracy: 57.51%
50	Validation loss: 0.804727	Best loss: 0.229844	Accuracy: 58.72%
51	Validation loss: 0.746395	Best loss: 0.229844	Accuracy: 65.44%
52	Validation loss: 0.539769	Best loss: 0.229844	Accuracy: 77.80%
53	Validat

In [205]:
#x = {1: (2,[1,2,3]), 3: (4,[3,4,5,6]), 4: (3,[9,5,4]), 5: (9,[9,5,4])}
#sorted_x = sorted(x.items(), key=lambda x:x[1],reverse=True)
#print (sorted_x)

[(5, (9, [9, 5, 4])), (3, (4, [3, 4, 5, 6])), (4, (3, [9, 5, 4])), (1, (2, [1, 2, 3]))]


In [166]:
sorted_fitness_dict

{0: (0.96516832068495817, [15, 10, 0.01]),
 1: (0.97139521307647403, [15, 10, 0.1]),
 2: (0.95913601868067722, [15, 20, 0.1]),
 3: (0.96730881494454179, [20, 10, 0.1]),
 4: (0.95835765713173771, [15, 20, 0.1])}

In [232]:
def generateChildren(sorted_fitness_dict,X_train1, y_train1,X_valid1,y_valid1,X_test1,y_test1):
    
    #Getting the last two keys of worst performing hyperparameters
    key1=list(sorted_fitness_dict.keys())[-1]
    key2=list(sorted_fitness_dict.keys())[-2]
    
    #print (key1)
    
    #getting parent keys (first best models) for producing a child
    parent_key1=list(sorted_fitness_dict.keys())[0]
    parent_key2=list(sorted_fitness_dict.keys())[1]
    
    #print (parent_key1)
    #print (type(parent_key1))
    
    #print (sorted_fitness_dict[parent_key1][1])
    
    #generating children from parent 1 and parent 2
    unmatched_list=[]
    list_parent_1=sorted_fitness_dict[parent_key1][1]
    list_parent_2=sorted_fitness_dict[parent_key2][1]
    
    print (list_parent_1)
    print (list_parent_2)
    
    #child1
    child_1_list=[]
    child_2_list=[]
    for i in range(len(list_parent_1)):
        
        if (list_parent_1[i]!=list_parent_2[i]):
            #unmatched_list.append(i)
            param_name=list(param.keys())[i]    #gets the name of the key in the param_dict where the index does not match, eg "n_neurons"
            par_list=param[param_name]   #par_list contains the values in that name
            ind=np.random.randint(0,len(par_list))   # generates random index number from that parameter name
            child_1_list.append(par_list[ind])   
            child_2_list.append(par_list[ind]) 
        else:
            child_1_list.append(list_parent_1[i])
            child_2_list.append(list_parent_1[i])
            
    print(child_1_list)
    #to speed up the process, if the child_1_list has already been computed, just use that value from the sorted_dict_list
    for k,v in sorted_fitness_dict.items():
        if (v[1]==child_1_list):
            child_1_cost=v[0]
        else:
            child_1_cost=getCost(X_train1, y_train1,X_valid1,y_valid1,X_test1,y_test1,child_2_list)
            
        if (v[1]==child_2_list):
            child_2_cost=v[0]
        else:
            child_2_cost=getCost(X_train1, y_train1,X_valid1,y_valid1,X_test1,y_test1,child_2_list)
    #child_2_cost=getCost(X_train1, y_train1,X_valid1,y_valid1,X_test1,y_test1,child_2_list)
    
    #adding the new two children by replacing them with the worst ones
    
    sorted_fitness_dict[key1]=(child_1_cost,child_1_list)
    sorted_fitness_dict[key2]=(child_2_cost,child_2_list)
    
    #typecasting the below statement by dict automatiicaly rearranges the values in ascending order based on keys
    sorted_fitness_list = sorted(sorted_fitness_dict.items(),  key=lambda x:x[1],reverse=True)  #returns a list
    
    for i in sorted_fitness_list:
        key=sorted_fitness_list[i][0]
        value=sorted_fitness_list[i][1]
        sorted_fitness_dict[key]=value
    
    return sorted_fitness_dict
    

In [2]:
generateChildren(sorted_fitness_dict,X_train1, y_train1,X_valid1,y_valid1,X_test1,y_test1)

In [173]:
sorted_fitness_dict

{0: (0.96516832068495817, [15, 10, 0.01]),
 1: (0.97139521307647403, [15, 10, 0.1]),
 2: (0.96166569371473054, [15, 10, 0.1]),
 3: (0.96730881494454179, [20, 10, 0.1]),
 4: (0.96302782642537454, [15, 10, 0.1])}

In [234]:
n_generations=3
population=6

param={'n_hidden_layers':[5,10,15,20],'n_neurons':[10,20,30,15],'learning_rate':[0.0001,0.001,0.01,0.1,1]}


sorted_fitness_dict=getTop_N_Fit(population)
for i in range(n_generations):
    print ('generation no:',i)
    
    #generating random parameters
    
    
    
    #removing samples with worst fitness and generating children
    sorted_fitness_dict=generateChildren(sorted_fitness_dict,X_train1, y_train1,X_valid1,y_valid1,X_test1,y_test1)
    
    print ('Dict values at generation no:',i)
    print (sorted_fitness_dict)

0	Validation loss: 0.341816	Best loss: 0.341816	Accuracy: 91.01%
1	Validation loss: 0.307190	Best loss: 0.307190	Accuracy: 91.28%
2	Validation loss: 0.398243	Best loss: 0.307190	Accuracy: 89.44%
3	Validation loss: 0.593868	Best loss: 0.307190	Accuracy: 78.26%
4	Validation loss: 0.684187	Best loss: 0.307190	Accuracy: 71.15%
5	Validation loss: 0.396820	Best loss: 0.307190	Accuracy: 89.37%
6	Validation loss: 0.543564	Best loss: 0.307190	Accuracy: 81.20%
7	Validation loss: 0.332144	Best loss: 0.307190	Accuracy: 91.52%
8	Validation loss: 0.338050	Best loss: 0.307190	Accuracy: 90.77%
9	Validation loss: 0.281040	Best loss: 0.281040	Accuracy: 92.34%
10	Validation loss: 0.261848	Best loss: 0.261848	Accuracy: 93.35%
11	Validation loss: 0.482828	Best loss: 0.261848	Accuracy: 76.70%
12	Validation loss: 0.484601	Best loss: 0.261848	Accuracy: 78.23%
13	Validation loss: 0.195486	Best loss: 0.195486	Accuracy: 95.15%
14	Validation loss: 0.207603	Best loss: 0.195486	Accuracy: 94.76%
15	Validation loss: 

37	Validation loss: 0.402780	Best loss: 0.205862	Accuracy: 79.12%
38	Validation loss: 0.480582	Best loss: 0.205862	Accuracy: 78.77%
39	Validation loss: 0.436078	Best loss: 0.205862	Accuracy: 83.31%
40	Validation loss: 3.275888	Best loss: 0.205862	Accuracy: 33.97%
41	Validation loss: 0.306263	Best loss: 0.205862	Accuracy: 91.59%
42	Validation loss: 0.199373	Best loss: 0.199373	Accuracy: 95.31%
43	Validation loss: 0.190517	Best loss: 0.190517	Accuracy: 95.66%
44	Validation loss: 1.289186	Best loss: 0.190517	Accuracy: 37.18%
45	Validation loss: 1.331232	Best loss: 0.190517	Accuracy: 32.72%
46	Validation loss: 1.583658	Best loss: 0.190517	Accuracy: 19.27%
47	Validation loss: 1.581042	Best loss: 0.190517	Accuracy: 20.60%
48	Validation loss: 1.610905	Best loss: 0.190517	Accuracy: 23.89%
49	Validation loss: 1.583603	Best loss: 0.190517	Accuracy: 20.60%
50	Validation loss: 1.596970	Best loss: 0.190517	Accuracy: 23.89%
51	Validation loss: 1.584650	Best loss: 0.190517	Accuracy: 20.95%
52	Validat

25	Validation loss: 0.092324	Best loss: 0.074404	Accuracy: 98.08%
26	Validation loss: 0.083122	Best loss: 0.074404	Accuracy: 97.77%
27	Validation loss: 0.072871	Best loss: 0.072871	Accuracy: 98.32%
28	Validation loss: 0.091993	Best loss: 0.072871	Accuracy: 98.01%
29	Validation loss: 0.104250	Best loss: 0.072871	Accuracy: 98.32%
30	Validation loss: 0.092477	Best loss: 0.072871	Accuracy: 98.05%
31	Validation loss: 0.072046	Best loss: 0.072046	Accuracy: 98.01%
32	Validation loss: 0.091952	Best loss: 0.072046	Accuracy: 98.01%
33	Validation loss: 0.078987	Best loss: 0.072046	Accuracy: 98.28%
34	Validation loss: 0.089436	Best loss: 0.072046	Accuracy: 98.01%
35	Validation loss: 0.137224	Best loss: 0.072046	Accuracy: 97.81%
36	Validation loss: 0.098751	Best loss: 0.072046	Accuracy: 97.73%
37	Validation loss: 0.103396	Best loss: 0.072046	Accuracy: 97.97%
38	Validation loss: 0.075644	Best loss: 0.072046	Accuracy: 98.51%
39	Validation loss: 0.099174	Best loss: 0.072046	Accuracy: 98.28%
40	Validat

7	Validation loss: 0.112907	Best loss: 0.074560	Accuracy: 97.73%
8	Validation loss: 0.091345	Best loss: 0.074560	Accuracy: 97.46%
9	Validation loss: 0.085484	Best loss: 0.074560	Accuracy: 97.65%
10	Validation loss: 0.080211	Best loss: 0.074560	Accuracy: 97.97%
11	Validation loss: 0.118857	Best loss: 0.074560	Accuracy: 97.30%
12	Validation loss: 0.087438	Best loss: 0.074560	Accuracy: 97.89%
13	Validation loss: 0.084605	Best loss: 0.074560	Accuracy: 97.85%
14	Validation loss: 0.075725	Best loss: 0.074560	Accuracy: 97.85%
15	Validation loss: 0.078562	Best loss: 0.074560	Accuracy: 97.62%
16	Validation loss: 0.114954	Best loss: 0.074560	Accuracy: 97.54%
17	Validation loss: 0.114267	Best loss: 0.074560	Accuracy: 98.01%
18	Validation loss: 0.090053	Best loss: 0.074560	Accuracy: 97.34%
19	Validation loss: 0.080613	Best loss: 0.074560	Accuracy: 98.05%
20	Validation loss: 0.091760	Best loss: 0.074560	Accuracy: 97.81%
21	Validation loss: 0.114324	Best loss: 0.074560	Accuracy: 97.50%
22	Validation

9	Validation loss: 0.123141	Best loss: 0.096661	Accuracy: 97.42%
10	Validation loss: 0.091894	Best loss: 0.091894	Accuracy: 97.69%
11	Validation loss: 0.102535	Best loss: 0.091894	Accuracy: 97.46%
12	Validation loss: 0.086887	Best loss: 0.086887	Accuracy: 97.22%
13	Validation loss: 0.128858	Best loss: 0.086887	Accuracy: 97.50%
14	Validation loss: 0.085459	Best loss: 0.085459	Accuracy: 97.73%
15	Validation loss: 0.093410	Best loss: 0.085459	Accuracy: 97.11%
16	Validation loss: 0.107087	Best loss: 0.085459	Accuracy: 97.11%
17	Validation loss: 0.095439	Best loss: 0.085459	Accuracy: 97.15%
18	Validation loss: 0.099962	Best loss: 0.085459	Accuracy: 97.50%
19	Validation loss: 0.125496	Best loss: 0.085459	Accuracy: 97.03%
20	Validation loss: 0.117665	Best loss: 0.085459	Accuracy: 97.69%
21	Validation loss: 0.098031	Best loss: 0.085459	Accuracy: 97.54%
22	Validation loss: 0.154966	Best loss: 0.085459	Accuracy: 97.26%
23	Validation loss: 0.104618	Best loss: 0.085459	Accuracy: 97.22%
24	Validati

32	Validation loss: 0.138448	Best loss: 0.080666	Accuracy: 97.73%
33	Validation loss: 0.112235	Best loss: 0.080666	Accuracy: 97.97%
34	Validation loss: 0.126586	Best loss: 0.080666	Accuracy: 97.85%
35	Validation loss: 0.199129	Best loss: 0.080666	Accuracy: 97.85%
36	Validation loss: 0.087656	Best loss: 0.080666	Accuracy: 98.16%
37	Validation loss: 0.094059	Best loss: 0.080666	Accuracy: 97.89%
38	Validation loss: 0.135952	Best loss: 0.080666	Accuracy: 97.69%
39	Validation loss: 0.129382	Best loss: 0.080666	Accuracy: 97.26%
40	Validation loss: 0.106241	Best loss: 0.080666	Accuracy: 97.89%
41	Validation loss: 0.128947	Best loss: 0.080666	Accuracy: 97.81%
42	Validation loss: 0.096874	Best loss: 0.080666	Accuracy: 97.93%
Early stopping!
0	Validation loss: 0.089311	Best loss: 0.089311	Accuracy: 97.26%
1	Validation loss: 0.072946	Best loss: 0.072946	Accuracy: 98.01%
2	Validation loss: 0.081267	Best loss: 0.072946	Accuracy: 97.81%
3	Validation loss: 0.087614	Best loss: 0.072946	Accuracy: 97.81

TypeError: list indices must be integers or slices, not tuple

In [186]:
#sorted_fitness_dict

{0: (0.968476357267951, [20, 10, 0.01]),
 1: (0.96944930920412531, [20, 10, 0.01]),
 2: (0.95699552442109359, [15, 10, 0.01]),
 3: (0.94784977622105471, [15, 10, 0.1]),
 4: (0.95991438022961662, [20, 10, 0.01]),
 5: (0.96477913991048847, [15, 10, 0.01]),
 6: (0.96069274177855613, [20, 10, 0.001]),
 7: (0.95446584938704027, [20, 10, 0.01]),
 8: (0.95894142829344231, [15, 20, 0.01]),
 9: (0.95446584938704027, [15, 20, 0.01])}

In [223]:
dict(sorted_fitness_dict)

{0: (0.97139521307647403, [15, 10, 0.01]),
 1: (0.89686709476551862, [15, 30, 0.01]),
 2: (0.9575792955827982, [20, 20, 0.1]),
 3: (0.96030356100408643, [20, 20, 0.001]),
 4: (0.9361743529869625, [15, 30, 0.01]),
 5: (0.96575209184666277, [20, 20, 0.1])}

In [1]:
sorted(sorted_fitness_dict.items(), key=lambda x:x[1],reverse=True)

In [224]:
from collections import OrderedDict
OrderedDict(sorted(sorted_fitness_dict.items()))

AttributeError: 'list' object has no attribute 'items'

In [231]:
sorted_fitness_dict[1][1]

(0.96575209184666277, [20, 20, 0.1])

In [ ]:
fo